In [ ]:
from analysis import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Qualisys and user defined functions

The goal of this tutorial is to use the wave probe and Qualisys data to demonstrate how to develop user defined functions and loops to speed up data analysis. 

First, let's define a python function to replicate the wave plot of the previous notebook. 

In [ ]:
def plot_wave(filename,start,end=1e3):
    (read_waves(filename)
     .query('time>@start & time<@end')
     .plot(x='time',title=filename))
plot_wave('../WaveProbe/run2.csv',start=100,end=110)
plt.show()

We have defined the function `plot_wave` which takes in three _arguments_: the `filename` as a string, and the `start` and `end` time for the plot window. This allows us to look at any file over any window of time without further copy-pasting. Play around with the argument values.

The only tricky bit of code above is the `@` symbol in front of the argument names inside the query string. This lets Pandas know they are arguments.

The next step is to loop over all the runs. This is done with a `for` loop and a `format` function to make each file name string.

In [ ]:
list_nums = [1,2,3,4,'5_6',7,'8_9',10]
for run in list_nums:
    filename = '../WaveProbe/run{}.csv'.format(run)
    plot_wave(filename,start=0,end=2e3)

We can see the first run looks like somethings gone wrong and we'll have to ignore that data. There are also a few spikes which indicate momentary errors in the data aquisition. This is very typical and so we should use robust metrics like [mad](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mad.html) rather than simply min/max. 

---

Next let's read the Qualisys data.

The qualisys data recorded motion tracking data for the Dynamic Positioning (_smarty_) and moored platforms during each run. I've created a function `read_qualisys` to get that data.

First let's see what the function help looks like

In [ ]:
help(read_qualisys)

So we need to give it a file name, but also an offset to tell the method which columns we want. 

In [ ]:
filename = '../Qualisys/run0002_6D.tsv'
smarty_offset = 19  # I think this is smarty
(read_qualisys(filename,smarty_offset)
 .plot(x='time',y='q_6'))
plt.show()

In [ ]:
filename = '../Qualisys/run0002_6D.tsv'
smarty_offset = 2  # I think this is for smarty
moored_offset = 19   # I think this is the first column for the moored platform
read_qualisys(filename,moored_offset).describe()

In [ ]:
read_qualisys(filename,smarty_offset).describe()

Each dataframe has the recorded location of each rigid body degree of freedom ($q_1\ldots q_6$) measured at 100 $Hz$. Note that we have correctly identified smarty and the moored platform because the smarty moves around a lot more (under it's own power) in these small waves.

Let's plot the heave.

In [ ]:
(read_qualisys(filename,moored_offset)
 .plot(x='time',y='q_3'))
plt.show()

This looks pretty good, but the mean must have something to do with the distance blow the qualisys cameras instead of the equilibrium height. Let's write a little function to zero the mean of the heave data.

In [ ]:
def zero_mean_heave(df): # df can be _any_ data frame
    df.q_3 = df.q_3-df.q_3.mean() # remove the mean
    return df

(read_qualisys(filename,moored_offset)
 .pipe(zero_mean_heave)
 .plot(x='time',y='q_3'))
plt.show()

We wrote a little function that takes the data as an input and returns the modified data. Then we included it in the pipeline with the `pipe` function. Pretty slick!

How about the smarty heave?

In [ ]:
(read_qualisys(filename,smarty_offset)
 .plot(x='time',y='q_3'))
plt.show()

Here we have more data aquisition problems. In this case, when Qualysis looses track of the vehicle it just returns $q=0$. But it would be much better to drop these values instead. Let's use `query` again.

In [ ]:
(read_qualisys(filename,smarty_offset)
 .query('q_3!=0') # select all points with non-zero heave
 .pipe(zero_mean_heave)
 .plot(x='time',y='q_3'))
plt.show()

This is much better, but now there are big gaps in the signal. Alternatively, we could use the `interp_bad` function in analysis.py.

For now, lets live with it and look at the horizontal plan motions for each platform.

In [ ]:
(read_qualisys(filename,moored_offset)
 .query('q_2!=0')
 .plot(x='q_1',y='q_2',title='moored'))

(read_qualisys(filename,smarty_offset)
 .query('q_2!=0')
 .plot(x='q_1',y='q_2',title='smarty'))
plt.show()

Notice the difference in scale! Here's a version showing both on top of each other, and I've looped over all the runs in the `range` 1--11. 

In [ ]:
@try_catch
def plot_horizontal(filename):
    moored = read_qualisys(filename,moored_offset).query('q_2!=0')
    smarty = read_qualisys(filename,smarty_offset).query('q_2!=0')
    plt.plot(moored.q_1,moored.q_2,label='moored')
    plt.plot(smarty.q_1,smarty.q_2,label='smarty')
    plt.title(filename)
    plt.legend()
    plt.show()

for run in range(1,11):
    filename = '../Qualisys/run{:04d}_6D.tsv'.format(run)
    plot_horizontal(filename)

We see that the moored object stays pretty weel fixed compared to smarty. We also see that we have a small naming convention problem in the dataset.